In [1]:
import os
import uuid
import torch
import torchvision.transforms as transforms
from PIL import Image
from tqdm import tqdm

In [2]:
# ⚠️ Set these paths manually before running
DATASET_FOLDER = "./Unsplitted_Ready_Sets/set_01_raw_class_balanced"  # The original dataset folder with 7 classes
AUGMENTED_FOLDER = "./Unsplitted_Ready_Sets/set_01_class_balanced_augs_applied"  # Where augmented images will be stored

# Emotion categories (Folder names must match)
EMOTION_CATEGORIES = ["Angry", "Disgust", "Fear", "Happy", "Neutral", "Sad", "Surprise"]

# Set augmentation parameters
IMAGE_SIZE = (260, 260)  # Target image size
IMAGES_PER_CLASS = 7382  # Number of images per class

In [3]:
# Define augmentation transformations
def get_augmentations():
    return [
        # First augmentation: Horizontal Flip + Random Crop
        transforms.Compose([
            transforms.RandomHorizontalFlip(p=1.0),
            transforms.RandomResizedCrop(IMAGE_SIZE, scale=(0.9, 1.0)),
            transforms.ToTensor(),
            # transforms.Normalize(mean=[0.5], std=[0.5])
        ]),
        # Second augmentation: Rotation + Brightness/Contrast Adjustment
        transforms.Compose([
            transforms.RandomRotation(degrees=15),
            transforms.ColorJitter(brightness=0.1, contrast=0.1),
            transforms.ToTensor(),
            # transforms.Normalize(mean=[0.5], std=[0.5])
        ]),
        # Third augmentation: Gaussian Noise + Color Jittering
        transforms.Compose([
            transforms.ToTensor(),
            transforms.Lambda(lambda x: x + torch.randn_like(x) * 0.02),
            transforms.ColorJitter(saturation=0.1),
            # transforms.Normalize(mean=[0.5], std=[0.5])
        ])
    ]

In [4]:
# Step 1: Create the "AUGMENTED_FOLDER" with emotion subfolders
def initialize_augmented_dataset():
    if not os.path.exists(AUGMENTED_FOLDER):
        os.makedirs(AUGMENTED_FOLDER)

    for emotion in EMOTION_CATEGORIES:
        emotion_folder = os.path.join(AUGMENTED_FOLDER, emotion)
        if not os.path.exists(emotion_folder):
            os.makedirs(emotion_folder)

In [5]:
# Step 2: Apply augmentations to images
def augment_class(emotion_category):
    source_folder = os.path.join(DATASET_FOLDER, emotion_category)
    target_folder = os.path.join(AUGMENTED_FOLDER, emotion_category)

    # Check if source folder exists
    if not os.path.exists(source_folder):
        print(f"❌ Skipping {emotion_category}: No such folder in dataset.")
        return

    # Get all images in the class folder
    image_paths = [
        os.path.join(source_folder, file) for file in sorted(os.listdir(source_folder))
        if file.lower().endswith((".jpg", ".jpeg", ".png"))
    ]

    # Get transformations
    augmentations = get_augmentations()

    print(f"📂 Augmenting {emotion_category}...")

    # Step 2.1: Process each image with all augmentations
    with tqdm(total=len(image_paths) * 3, desc=f"Augmenting {emotion_category}") as pbar:
        for img_path in image_paths:
            try:
                img = Image.open(img_path).convert("RGB")  # Keep color for augmentation

                for j, transform in enumerate(augmentations):
                    augmented_img = transform(img)
                    augmented_img = transforms.ToPILImage()(augmented_img)

                    # Ensure unique filenames
                    unique_name = f"aug_{j}_{str(uuid.uuid4())[:8]}_{os.path.basename(img_path)}"
                    target_path = os.path.join(target_folder, unique_name)
                    augmented_img.save(target_path)

                    pbar.update(1)

            except Exception as e:
                print(f"⚠️ Error processing {img_path}: {e}")
                pbar.update(1)

    print(f"✅ Augmentation completed for {emotion_category}.")

In [6]:
# Step 3: Run the augmentation process
if __name__ == "__main__":
    initialize_augmented_dataset()

    # Process each emotion category
    for category in EMOTION_CATEGORIES:
        augment_class(category)

    print(f"\n✅ Dataset augmentation completed! Augmented images are stored in: {AUGMENTED_FOLDER}")

📂 Augmenting Angry...


Augmenting Angry: 100%|██████████| 22146/22146 [12:45<00:00, 28.93it/s]


✅ Augmentation completed for Angry.
📂 Augmenting Disgust...


Augmenting Disgust: 100%|██████████| 22146/22146 [12:29<00:00, 29.55it/s]


✅ Augmentation completed for Disgust.
📂 Augmenting Fear...


...

✅ Augmentation completed for Neutral.
📂 Augmenting Sad...


Augmenting Sad: 100%|██████████| 22146/22146 [12:08<00:00, 30.42it/s] 


✅ Augmentation completed for Sad.
📂 Augmenting Surprise...


Augmenting Surprise: 100%|██████████| 22146/22146 [15:23<00:00, 23.97it/s]

✅ Augmentation completed for Surprise.

✅ Dataset augmentation completed! Augmented images are stored in: ./Unsplitted_Ready_Sets/set_01_class_balanced_augs_applied
